# INTRO 
This notebook details 5G scenario preparation, data collection, processing, and offline AI model training. 

For the preparation and data collection sections it is assumed that readers will already have working 5G infrastructure. 

Infrastructure includes:
- A working Open5GS core with Prometheus enabled 
- 6 Open5GS UPFs (if you want to replicate our scenario exactly)
- An Amarisoft gnb and ue sim 
- A deployment of ONAP v13.0.0
- VESPA for Prometheus - VES event conversion 

![Full testbed](./artifacts/experiment_figures/Full-Core.png)

In the above image you can see the interfaces connecting different portions of the testbed. The Amarisoft callbox hosts the eNB which must be registered with the Open5GS core network. Each of the UPFs in the core network must be routed to the control plane in the SMF configuration. Open5GS core metrics are supplied to the ONAP message router through VESPA which sends Prometheus data to the ONAP VES-collector. 

# Setup 

## Open5GS 
The artifacts of our deployment are included in the Open5GS-Artifacts directory. These artifacts can be deployed in your cluster to set up the SMF, AMF, and NSSF in our configuration. The SMF and AMF HPA scaling rules are also included. 

### UPF Modifications 
The Open5GS deployment needs to be modified to support multiple slices. For our core network we deployed each UPF on a separate VM and specified routing rules to enable communication with the core network. The outline of our routing rules:

```
sudo ip tuntap add name ogstun mode tun
sudo ip addr add 10.46.0.1/16 dev ogstun    # 10.46.0.1/16 is the subnet used for upf sessions 
sudo ip link set ogstun up
sudo iptables -t nat -A POSTROUTING -s 10.46.0.0/16 ! -o ogstun -j MASQUERADE
./install/bin/open5gs-upfd
sudo ip route add UPF_IP via CORE_IP
```

Additionally, we needed to edit the Open5GS code to enable counters which had been previously disabled due to performance issues. See the following dif:
```
diff --git a/src/upf/gtp-path.c b/src/upf/gtp-path.c
index d2fef084d..1676f3bce 100644
--- a/src/upf/gtp-path.c
+++ b/src/upf/gtp-path.c
@@ -223,7 +223,7 @@ static void _gtpv1_tun_recv_common_cb(
      * It should not be used on the UPF/SGW-U data plane
      * until this issue is resolved.
      */
-#if 0
+#if 1
     upf_metrics_inst_global_inc(UPF_METR_GLOB_CTR_GTP_OUTDATAPKTN3UPF);
     upf_metrics_inst_by_qfi_add(pdr->qer->qfi,
         UPF_METR_CTR_GTP_OUTDATAVOLUMEQOSLEVELN3UPF, recvbuf->len);
@@ -390,7 +390,7 @@ static void _gtpv1_u_recv_cb(short when, ogs_socket_t fd, void *data)
          * It should not be used on the UPF/SGW-U data plane
          * until this issue is resolved.
          */
-#if 0
+#if 1
         upf_metrics_inst_global_inc(UPF_METR_GLOB_CTR_GTP_INDATAPKTN3UPF);
         upf_metrics_inst_by_qfi_add(header_desc.qos_flow_identifier,
                 UPF_METR_CTR_GTP_INDATAVOLUMEQOSLEVELN3UPF, pkbuf->len);
```


### UE Definitions on Open5GS 
The `reset_database.sh` script can be used to add the same UE configurations to the Open5GS dbctl. You need to provide it with the name of your populate pod. It is *extremely* unoptimized and slow because it uses the open5gs-dbctl commands to add UE entries rather than going straight to mongodb. 

## VESPA
See the VESPA github to setup VESPA https://github.com/nokia/ONAP-VESPA. Our ves-agent.yml configuration is included in the Open5GS-Artifacts directory. **Modify it to reference your IP addresses** if you want to use the same VES events as us. 

Configuring VESPA:
```
sudo cp ves-agent.yml /etc/ves-agent/
```

Once the VES-collector is running on ONAP, VSEPA can be used to send Prometheus data collected from the Open5GS core to the ONAP VES-collector. VESPA is activated from the ves-agent directory in the VESPA project. 
```
./ves-agent
```

The VES agent will send POSTs to the address specified in ves-agent.ml and if successful will return a 200 response. 

## Amarisoft 
The `Amarisoft` directory contains the scenarios we used for data collection. These scenarios can be loaded into Amarisoft by replacing the default `ue.cfg` file and running `systemctl restart lte`. 

You can make your own scenarios by either manually scripting "sim_events" similar to `Amarisoft/ue-list.json` or by using the Amarisoft UE SIM BOX GUI. 

DSM attack scenarios can be created from benign scenarios by taking the `ue-list.json` file you wish to modify and using `manual_dsm.py`. This python file assumes that you used the same UE ids and configuration as our deployment and **will not** work on arbitrary scenarios. 

Amarisoft uses collectd to scrape UE metrics from the callbox api. 
1. Install collectd on the Amarisoft UE SIM BOX VM
2. Add `Amarisoft/Amarisoft UE data collection/collectd.conf` to `/etc/collectd` 
3. Add `Amarisoft/Amarisoft UE data collection/types.db` to `/etc/collectd/plugins/`
4. Add `Amarisoft/Amarisoft UE data collection/enb_stats.py` to `/etc/collectd/plugins`
5. Add `Amarisoft/Amarisoft UE data collection/enb_utils.py` to `/etc/collectd/plugins`
6. Add `Amarisoft/Amarisoft UE data collection/enb_list.cfg` to `/etc/collectd/plugins/`

# Data Collection 
1. Start the scenario from Amarisoft using `systemctl restart lte`

    ## Open5GS 
    1. Activate the VES-Agent using `./ves-agent` wherever VESPA is located 
    2. Run `consumer.py` to begin collecting a VES events from the ONAP message router in JSON format

    ## Amarisoft 
    1. Start the collectd plugin using `systemctl start collectd` 



In [ ]:
# WARNING: THIS CELL IS FOR DEMONSTRATION PURPOSES ONLY AND WILL THROW A TIMEOUT ERROR IF CANNOT CONNECT TO THE CORE NETWORK

from consumer import *

onap_ip = '127.0.0.1'
message_router_port = '1234'
scenario_duration = 60 # Given in seconds 
polling_interval = 15 # Given in seconds. Is determined by the configuration of Prometheus on the core network 
save_file = 'raw_data/core-ves-data.json'

ves_consumer_main(onap_ip, message_router_port, scenario_duration, polling_interval, save_file)

# Data Processing 
These are the metrics we collected for our AI model:
## Open5GS - Prometheus -> VESPA -> VES-Collector 
- Number of UE RAN connections 
- Number of AMF sessions per slice
- Number of PDU creation requests per slice
- Number of successful PDU session creations per slice 
- Number of N4 interface session establishment requests (between SMF and UPF) per slice
- Number of successful N4 establishments per slice 
- Number of failed N4 establishments per slice 
- CPU resources requested per VNF 
- UE registration initiaions with the AMF
- Successful UE registrations with the AMF
- UPF uplink packet count per slice 
- UPF downlink packet count per slice 
- Number of SMF sessions per slice 
- Number of UEs active on the Open5GS core network 
- Container CPU use (CPU seconds)
- AMF registered UE subscribers per slice 
- Number of PCF association requests per slice 
- Number of PCF sessions per slice 

## Amarisoft UE data (Collected for each UE)
- Number of PDU sessions with each slice 
- Downlink rx 
- Downlink retransmissions 
- Downlink errors
- Uplink tx
- Uplink retransmissions
- Uplink datarate
- Downlink datarate 

In [ ]:
# NRves2csv handles ves-csv conversion and processing UE data into a single csv 
from NRves2csv import *

In [ ]:
# Provide the paths to raw data and where you want to save it 

# Core data comes from a json file that lists all the VES events files to pull data from. 
# The json file specifies which scenarios are malicious and which slice is under attack
core_data = Path('raw_data/core-data/ves_files_list.json') 

# UE data comes from a directory that has a sub-directory for each UE 
bengin_ue_dir = Path('raw_data/ue-data/ue-benign-samples')
malicious_ue_dir = Path('raw_data/ue-data/ue-malicious-samples')

# Define the save files 
save_file_core = Path('data/core-dataset.csv')
save_file_slice = Path('data/slice-dataset.csv')
save_file_ue = Path('data/ue-dataset.csv')

In [ ]:
# Process the raw core data into dataframes 

# Core Dataframe
core_data = json.loads(core_data.read_text(encoding="UTF-8"))
ves_files = list(core_data.keys())
mal_ids = [list(targets.values())[0] for targets in core_data.values()]
target_slice_sds = [list(targets.values())[1] for targets in core_data.values()]
core_frames = [ves2csv(Path(path), mal_id) for path, mal_id in zip(ves_files, mal_ids)]
# Combined and save core data to csv
core_df = combine_csv(core_frames, save_file_core)

# Slice Dataframe 
# Specify the SST-SD of each slice as well as the IP address of the UPF VM 
# These labels are deployment specific and originate from the Prometheus -> VESPA -> ONAP data pipeline 
slice_labels = [('1-111111','10.1.0.138:9090'), 
                ('1-222222','10.1.0.201:9090'), 
                ('2-333333','10.1.0.228:9090'), 
                ('2-444444','10.1.0.33:9090'), 
                ('3-555555','10.1.0.76:9090'), 
                ('3-666666','10.1.0.232:9090')]
# Split the core data by slice 
slice_frames = [split_slices(core_frame, slice_labels, target_slice_sd) for core_frame, target_slice_sd in zip(core_frames, target_slice_sds)]
# Combine slice specific data frames and save to csv
slice_df = combine_csv(slice_frames, save_file_slice)


In [ ]:
# Process the raw UE data into dataframes 

# Specify data filters for UE data. EX: ['09-24'] will only include data from files with '09-24' in the title 
# benign_data_filter = ['09-24'] 
# malicious_data_filter = ['09-24']
benign_data_filter = None 
malicious_data_filter = None

# Load the list of malicious UE IMSIs: 
mal_ue_file = Path('artifacts/malicious_ues.json')
mal_ue_info = json.loads(mal_ue_file.read_text(encoding="UTF-8"))
mal_ue_imsis = mal_ue_info['mal_ue_imsis']

# UE Dataframes
benign_ue_df = make_ue_df(bengin_ue_dir, benign_data_filter)
malicious_ue_df = make_ue_df(malicious_ue_dir, malicious_data_filter, mal_ue_list = mal_ue_imsis, malicious = True)

# Combine benign/malicious data and save to csv
full_ue_df = combine_csv([benign_ue_df, malicious_ue_df], saveFile=save_file_ue)

# AI Datasets + Model 
Now that the raw data from the Open5GS core network and Amarisoft UEs has been processed into csv files we can create torch datasets and train AI models. 

TLDR:
- Min-Max Scale
- Sequences of (24) samples [[sampl1], [sample2], ... [sample24]]
- Batched inputs to the torch model (batch size 100) during training 
- Batch size of 1 during testing 
    - Must be done to plot reconstruction loss of individual samples 

![Anomaly Detection Framework](./artifacts/experiment_figures/Anomaly-Detector.png)


## Model Inputs 
We used 1D-CNN based autoencoders to capture the time dependency of the DSM attack. Each AI model takes a sequence of samples as an input. We used 24 samples per sequence to capture ~2min of data (at a sample rate of 5s) per inference made by the model. This sequence length was chosen in accordance with our DSM attack parameters which repeated at a frequency of ~2min. Each scenario we ran was 30min long. 

## Preprocessing 
### Scaling 
First, the UE and Core metrics have a wide scale (datarate could be in the thousands vs. # of PDU sessions which will be in the single digits) that needs to be standardized. We apply a Min-Max scaler to each feature so that larger features are not weighted more heavily. 

### Time-Series Sequences 
Sequences of 24 samples (can be modified by changing frame_size) are created by iterating through dataframes loaded from the processed csv files. Sequences of samples are created by walking through the csv file sample-by-sample and constructing a list of 24 samples. This means that each sample can contribute to multiple sequences. The list of samples is then cast to a float32 tensor using torch.stack(sample_frame).to(torch.float32). The first samples timestamp is set as the starting point and if the last sample in the expected sequence has a vastly different timestamp from what is expected the sequence is discarded. This prevents the data from different UEs, slices, or experiments from being merged together.  

#### Example time series: 
- Time series 1: [[sample1], [sample2], [sample3], ... [sample24]] 
- Time series 2: [[sample2], [sample3], [sample4], ... [sample25]]

## Training 
During model training the Autoencoder expects to only see benign data. This way, the autoencoder can be used to identify OOD data by looking at the reconstuction loss of samples. Malicious data should have a higher reconstruction loss because the Autoencoder has not seen similar data before. 

## Testing 
Benign and malicious data is included in the test set. The batch size **MUST BE SET TO 1** for the test loop in order to plot reconstruction loss. Batch size 1 is inconvenient for lots of data, but allows us to plot the reconstruction loss of every individual sample passed to the model. If we did a larger batch size the reconstruction loss would be calculated as the average over a set of samples and may lump benign and malicious data together.  

# Slice Attack Dataset Creation 

In [ ]:
from NRTimeSeriesML import *
import pickle

In [ ]:
# Setup 
device = torch_setup()

# Set the frame size for the DSM detection dataset 
# Data was collected at 5s intervals and the DSM attack we applied looped at ~2min intervals
# We selected a frame size of 24 to capture the full DSM loop: 24 (samples) * 5 (seconds / sample)  = 120 seconds 
frame_size = 24

In [ ]:
# Load the slice data into a dataframe 
slice_df = pd.read_csv('data/slice-dataset.csv')

# Split the dataframe into benign/malicious 
benign_slice = slice_df.loc[slice_df['label'] == 0]
malicious_slice = slice_df.loc[slice_df['label'] == 1]

# Setup a the data transformation - MinMaxScaler because the 5G data has large difference in scale across features 
transform_pipeline = Pipeline([('scaler', preprocessing.MinMaxScaler())])

# Create the time-aware dataset (dataset id is used to shape features correctly 0: core dataset 1: slice dataset 2: ue dataset)
slice_benign_dataset = NRTimeDataset(df=benign_slice, frame_length = frame_size, transform=transform_pipeline, dataset_id=1, fit=True)
slice_mal_dataset = NRTimeDataset(df=malicious_slice, frame_length = frame_size, transform=transform_pipeline, dataset_id=1, fit=True)

# Save the benign fit slice transformation for live data inference in the future 
pickle.dump(slice_benign_dataset.transform, open('artifacts/slice_transform.pkl', 'wb'))

In [ ]:
# Train-test split benign data 
slice_benign_split = torch.utils.data.random_split(slice_benign_dataset, [0.6,0.2,0.2])
slice_train_dataset, slice_val_dataset, slice_test_dataset = slice_benign_split

# All malicious data goes into the test set - The autoencoder is trained on BENIGN DATA ONLY! 
slice_test_dataset = slice_mal_dataset + slice_test_dataset

# Create data loaders 
slice_train_dataloader = DataLoader(slice_train_dataset, batch_size=100, shuffle=True)
slice_test_dataloader = DataLoader(slice_test_dataset, batch_size=1, shuffle=False) # Batch size 1 to track reconstruction loss of every sample 

# Slice Attack Detection Model 

In [ ]:
# Define the CNN filter sizes for the autoencoder encoder and decoder 
filter_sizes = [8, 4]

# Model definition - slice
CNNautoencoder_slice = ConvAutoencoder(n_features = slice_benign_dataset.samples.size()[1], filter_sizes=filter_sizes)
CNNautoencoder_slice.to(device)

In [ ]:
# PyTorch Lightning model instantiation - Information about training will be output to the lightning_logs dir
loss_fn = nn.MSELoss(reduction='mean')
NR_Autoencoder_slice = LitTimeAutoencoder(CNNautoencoder_slice, loss_fn)

In [ ]:
# Train and save the model 
# Autoencoder training 
trainer_slice = L.Trainer(limit_train_batches=100, max_epochs=100, callbacks=[AutoencoderReconstructionLoss()])
trainer_slice.fit(model=NR_Autoencoder_slice, train_dataloaders=slice_train_dataloader) # Model is the lightning module here 
# Save the slice state_dict
torch.save(CNNautoencoder_slice.state_dict(), 'artifacts/slice_state_dict')

In [ ]:
# Plot the reconstruction loss on test samples 
trainer_slice.test(NR_Autoencoder_slice, dataloaders=slice_test_dataloader)
plot_reconsctruction_loss(NR_Autoencoder_slice.reconstruction_loss)

# UE Dataset Creation 

In [ ]:
# Load the UE data into a dataframe 
ue_df = pd.read_csv('data/ue-dataset.csv')

# Split into bengin/malicious samples 
benign_ue = ue_df.loc[ue_df['label'] == 0]
malicious_ue = ue_df.loc[ue_df['label'] == 1]

# Create data transform + time series datasets 
transform_pipeline = Pipeline([('normalizer', preprocessing.Normalizer()), ('scaler', preprocessing.MinMaxScaler())])
ue_benign_dataset = NRTimeDataset(df=benign_ue, frame_length = frame_size, transform=transform_pipeline, dataset_id = 2)
ue_mal_dataset = NRTimeDataset(df=malicious_ue, frame_length = frame_size, transform=ue_benign_dataset.transform, dataset_id = 2, fit=False)

# Save the fit UE data transform for live data inference later 
pickle.dump(ue_benign_dataset.transform, open('artifacts/ue_transform.pkl', 'wb'))

In [ ]:
# Train-test split the benign data 
ue_benign_split = torch.utils.data.random_split(ue_benign_dataset, [0.6,0.2,0.2])
ue_train_dataset, ue_val_dataset, ue_test_dataset = ue_benign_split

# All malicious data goes in the test set - The autoencoder is trained on BENIGN DATA ONLY!
ue_test_dataset = ue_mal_dataset + ue_test_dataset

In [ ]:
# UE Data Loaders 
ue_train_dataloader = DataLoader(ue_train_dataset, batch_size=100, shuffle=True)
ue_test_dataloader = DataLoader(ue_test_dataset, batch_size=1, shuffle=False)

# UE Detection Model 

In [ ]:
# Define the CNN filter sizes for the autoencoder encoder and decoder 
filter_sizes = [8, 4]

# Model definition - UE
CNNautoencoder_ue = ConvAutoencoder(n_features = ue_benign_dataset.samples.size()[1], filter_sizes=filter_sizes)
CNNautoencoder_ue.to(device)

In [ ]:
# PyTorch Lightning model instantiation - Information about training will be output to the lightning_logs dir
loss_fn = nn.MSELoss(reduction='mean')
NR_Autoencoder_ue = LitTimeAutoencoder(CNNautoencoder_ue, loss_fn)

In [ ]:
# Train and save the model 
# Autoencoder training 
trainer_ue = L.Trainer(limit_train_batches=100, max_epochs=100, callbacks=[AutoencoderReconstructionLoss()])
trainer_ue.fit(model=NR_Autoencoder_ue, train_dataloaders=ue_train_dataloader) # Model is the lightning module here 

# Save the ue state_dict
torch.save(CNNautoencoder_ue.state_dict(), 'artifacts/ue_state_dict')

In [ ]:
# Plot UE model reconstruction loss 
trainer_ue.test(NR_Autoencoder_ue, dataloaders=ue_test_dataloader)
plot_reconsctruction_loss(NR_Autoencoder_ue.reconstruction_loss)

# Model Statistics 
Autoencoder-based anomaly detection is based on the difference in reconstruction loss between benign and malicious samples. To determine the difference between anaomalous and benign samples we first must calculate the reconstruction loss statistics of benign samples. 

In [ ]:
import seaborn

In [ ]:
# Slice Model 

# Get the mean and std of reconstruction loss for benign and malicious samples 
slice_benign_loss_list = [loss for loss, label in NR_Autoencoder_slice.reconstruction_loss if label == 0]
slice_malicious_loss_list = [loss for loss, label in NR_Autoencoder_slice.reconstruction_loss if label == 1]
slice_mean, slice_std = loss_statistics(slice_benign_loss_list)

# Calculate an example m-distance based on loss statistics 
m_dist = sample_distance(slice_benign_loss_list[0], slice_mean, slice_std)
print('Slice model statistics: ', slice_mean, slice_std)
print('Sample benign m-distance: ', m_dist)

slice_threshold_metrics = {'mean': slice_mean, 'std': slice_std}

In [ ]:
# Set the anomaly detection threshold based on some number of stds 
slice_detection_threshold = slice_mean + 3*slice_std

# Calculate confusion matrix stats based on the loss statistics 
n_fp = len([loss for loss in slice_benign_loss_list if loss >= slice_detection_threshold])
n_tp = len([loss for loss in slice_malicious_loss_list if loss >= slice_detection_threshold])
n_fn = len(slice_malicious_loss_list) - n_tp
n_tn = len(slice_benign_loss_list) - n_fp

print('True positives: ', n_tp)
print('False positives: ', n_fp)
print('True negatives: ', n_tn)
print('False negatives: ', n_fn)

In [ ]:
font = {'size':15}
plt.rc('font', **font)

total_mal = n_tp + n_fn 
total_benign = n_fp + n_tn

fig, axes = plt.subplots(3, 1, figsize=(8,16), sharex=True, sharey=True)
fig.supxlabel('Predicted Label')
fig.supylabel('True Label')
fig.suptitle('Slice Attack Detection Confusion Matricies')
for n in range(3):
    # Set the anomaly detection threshold based on some number of stds 
    slice_detection_threshold = slice_mean + (n+1)*slice_std

    # Calculate confusion matrix stats based on the loss statistics 
    n_fp = len([loss for loss in slice_benign_loss_list if loss >= slice_detection_threshold])
    n_tp = len([loss for loss in slice_malicious_loss_list if loss >= slice_detection_threshold])
    n_fn = len(slice_malicious_loss_list) - n_tp
    n_tn = len(slice_benign_loss_list) - n_fp

    axes[n].set_title('Slice Attack Detection - Mean + {} Std. Detection'.format(n))

    # Plot the confusion matrix for visualization 
    [[n_tp,n_fp],[n_tn,n_fn]]
    cm = np.array([[n_tp/total_mal,n_fn/total_mal],[n_fp/total_benign,n_tn/total_benign]])
    x_labels = ['Malicious', 'Benign']
    y_labels = ['Malicious', 'Benign']
    s = seaborn.heatmap(cm, xticklabels=x_labels, yticklabels=y_labels, annot=True, ax=axes[n])
    # s.set(xlabel='Slice Predicted Label', ylabel='Slice True Label')
    axes[n].set_title("Threshold = Mean reconstruction \n loss + {} Std. Deviation".format(n+1))

figure = s.get_figure()    
figure.savefig('plots/Slice_CM_detection_threshold-{}.png'.format(slice_detection_threshold), dpi=400)

In [ ]:
# UE model 

# Get the mean and std of reconstruction loss for benign and malicious samples 
ue_benign_loss_list = [loss for loss, label in NR_Autoencoder_ue.reconstruction_loss if label == 0]
ue_malicious_loss_list = [loss for loss, label in NR_Autoencoder_ue.reconstruction_loss if label == 1]
ue_mean, ue_std = loss_statistics(ue_benign_loss_list)
ue_mal_mean, ue_mal_std = loss_statistics(ue_malicious_loss_list)

# Calculate an example m-distance based on loss statistics 
m_dist = sample_distance(ue_malicious_loss_list[0], ue_mean, ue_std)
print('UE model statistics: ', ue_mean, ue_std, m_dist)
print('UE malicious statistics: ', ue_mal_mean, ue_mal_std)

ue_threshold_metrics = {'mean': ue_mean, 'std': ue_std}

In [ ]:
# Set the anomaly detection threshold based on some number of stds 

# For UE detection we want to set a high threshold - It is bad to ban a legitimate user so we should only treat the most anomalous
# samples as malicious
ue_detection_threshold = ue_mean + 3*ue_std

# TP, FP, TN, FN stats using reconstruction loss as a threshold metric 
n_fp = len([loss for loss in ue_benign_loss_list if loss >= ue_detection_threshold])
n_tp = len([loss for loss in ue_malicious_loss_list if loss >= ue_detection_threshold])
n_fn = len(ue_malicious_loss_list) - n_tp
n_tn = len(ue_benign_loss_list) - n_fp

print('True positives: ', n_tp)
print('False positives: ', n_fp)
print('True negatives: ', n_tn)
print('False negatives: ', n_fn)

In [ ]:
# Plot the confusion matrix for visualization 
[[n_tp,n_fp],[n_tn,n_fn]]
cm = np.array([[n_tp,n_fn],[n_fp,n_tn]])
x_labels = ['Malicious', 'Benign']
y_labels = ['Malicious', 'Benign']
s = seaborn.heatmap(cm, xticklabels=x_labels, yticklabels=y_labels, annot=True)
s.set(xlabel='UE Predicted Label', ylabel='UE True Label')
plt.title("UE Attack Detection - Mean + 3 Std. Detection")

figure = s.get_figure()    
figure.savefig('plots/UE_CM_detection_threshold-{}.png'.format(ue_detection_threshold), dpi=400)

In [ ]:
font = {'size':15}

total_mal = n_tp + n_fn 
total_benign = n_fp + n_tn

plt.rc('font', **font)
fig, axes = plt.subplots(3, 1, figsize=(8,16), sharex=True, sharey=True)
fig.supxlabel('Predicted Label')
fig.supylabel('True Label')
fig.suptitle('UE Attack Detection Confusion Matricies')
for n in range(3):
    # Set the anomaly detection threshold based on some number of stds 
    ue_detection_threshold = ue_mean + (n+1)*ue_std

    # TP, FP, TN, FN stats using reconstruction loss as a threshold metric 
    n_fp = len([loss for loss in ue_benign_loss_list if loss >= ue_detection_threshold])
    n_tp = len([loss for loss in ue_malicious_loss_list if loss >= ue_detection_threshold])
    n_fn = len(ue_malicious_loss_list) - n_tp
    n_tn = len(ue_benign_loss_list) - n_fp

    # Plot the confusion matrix for visualization 
    [[n_tp,n_fp],[n_tn,n_fn]]
    cm = np.array([[n_tp/total_mal,n_fn/total_mal],[n_fp/total_benign,n_tn/total_benign]])
    x_labels = ['Malicious', 'Benign']
    y_labels = ['Malicious', 'Benign']
    s = seaborn.heatmap(cm, xticklabels=x_labels, yticklabels=y_labels, annot=True, ax=axes[n])
    # s.set(xlabel='UE Predicted Label', ylabel='UE True Label')
    axes[n].set_title("Threshold = Mean reconstruction \n loss + {} Std. Deviation".format(n+1))


figure = s.get_figure()    
figure.savefig('plots/UE_CM_detection_threshold-{}.png'.format(ue_detection_threshold), dpi=400)

In [ ]:
# Save the detection parameters to the artifacts folder 
pickle.dump(slice_threshold_metrics, open('artifacts/slice_threshold_metrics.pkl', 'wb'))
pickle.dump(ue_threshold_metrics, open('artifacts/ue_threshold_metrics.pkl', 'wb'))